# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответь на четыре вопроса:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**
1. Обзор данных
2. Предобработка данных
3. Ответы на вопросы
4. Общий вывод

## 1. Обзор данных

In [1]:
import pandas as pd
from math import nan
from pymystem3 import Mystem
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()
print('\nchildren:',data['children'].unique(),'\n')
print('dob_years:',data['dob_years'].unique(),'\n')
print('education:',data['education'].unique(),'\n')
print('education_id:',data['education_id'].unique(),'\n')
print('family_status:',data['family_status'].unique(),'\n')
print('family_status_id:',data['family_status_id'].unique(),'\n')
print('gender:',data['gender'].unique(),'\n')
print('income_type:',data['income_type'].unique(),'\n')
print('debt:',data['debt'].unique(),'\n')
#print('purpose:',data['purpose'].unique(),'\n')
display(data.head(5))
display(data.tail(5))
display(data.sample(5 , random_state = 42))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

children: [ 1  0  3  2 -1  4 20  5] 

dob_years: [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75] 

education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Вы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
423,0,-191.167760,56,среднее,1,женат / замужем,0,M,сотрудник,0,138653.748793,автомобили
3522,0,-2319.817259,58,среднее,1,Не женат / не замужем,4,F,сотрудник,0,123152.627177,покупка жилья
8760,1,-2990.578297,34,высшее,0,гражданский брак,1,M,компаньон,0,232380.737167,свадьба
20695,0,-926.209452,28,среднее,1,Не женат / не замужем,4,F,сотрудник,0,210617.086601,получение высшего образования
4351,0,-2524.302106,42,СРЕДНЕЕ,1,женат / замужем,0,F,сотрудник,0,78487.540219,сделка с автомобилем


In [3]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


**Вывод**

Столбцы days_employed и total_income имеют пропуски, причем одинаковое количество.

Столбец children иммет явные неккореткные значения '-1' и возможнные неккоректные значения '20'(если таких значений мало в идеале одно, это может быть правдой но если их много то это явно ошибка).

Столбец days_employed содержит данные о общем стаже работы клиентов, соответственно наличие в нем отрицательных значений является неккоректным.

Столбец dob_years имеет неккоректные значения '0'.

Столбец education имеет записи одних и тех же категорий в разном регистре.

В столбце family_status одна категория выбивается из общего стиля нижнего регистра.

В столбце gender имеются пропуски.

Столбцы education и family_status имеют по сути дублирущие столбцы с id возможно создается избыточность(хотя и без расшифровки разумеется id не информативны).

## Предобработка данных

### Обработка пропусков

Мы имеем пропуски в трех столбцах, в столбце gender они неявные(заполнены значениями 'XNA') и для ответа на вопросы нам этот столбец не нужен, по этому он останется как есть. Столбцы days_employed и total_income количественные переменные, соответственно, их можно заполнить характерными для выборок значениями. Но, во-первых, количество пропусков одинаковое, возможно есть какая-то корреляция с другими признаками. Во-вторых, в столбце days_employed, есть отрицательные значения.

In [4]:
display(data[data['days_employed'].isna() == True])# явной зависимости от признаков не обнаруженно

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [5]:
print(data[data['days_employed'] == 0]['days_employed'].count()) #проверяем есть ли нули
display(data[data['days_employed'] < 0])

0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Отрицатльных значний не просто много, их большинство. На первый взгляд выделить какую то взаимосвязь с другими признами нельзя.

In [6]:
display(data[data['days_employed'] > 0])
display(data[data['days_employed'] > 0]['income_type'].unique())
display(data[data['income_type'] == 'безработный'])

print('\nОсновные метрики стажа пенсионеров и безработных:\n')
print('Минимум: ',data[data['days_employed'] > 0]['days_employed'].min())
print('Среднее: ',data[data['days_employed'] > 0]['days_employed'].mean())
print('Максимум:',data[data['days_employed'] > 0]['days_employed'].max(),'\n')

print('Минимальный стаж в годах(если указаны дни): ',data[data['days_employed'] > 0]['days_employed'].min()/365)

print('Минимальный стаж в годах(если указаны часы):',data[data['days_employed'] > 0]['days_employed'].min()/365/24)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


array(['пенсионер', 'безработный'], dtype=object)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью



Основные метрики стажа пенсионеров и безработных:

Минимум:  328728.72060451825
Среднее:  365004.3099162686
Максимум: 401755.40047533 

Минимальный стаж в годах(если указаны дни):  900.6266317932007
Минимальный стаж в годах(если указаны часы): 37.526109658050025


От сюда следует что положительные значения в наличии только у пенсионеров, и безработных(безработных всего двое, можно сказать просто пенсионерам), но отсюда вытекает еще одна проблема: положительные трудовые стажи явно не корректные, так как столбец показывает количество дней, то чтобы получить минимальный из данного списка стаж необходимо проработать более 900 лет, видимо были указаны часы а не дни.

In [7]:
print('\nОсновные метрики стажа пенсионеров и безработных после перевода часов в дни:\n')
print('Минимум: ',data[data['days_employed'] > 0]['days_employed'].min()/24)
print('Среднее: ',data[data['days_employed'] > 0]['days_employed'].mean()/24)
print('Максимум:',data[data['days_employed'] > 0]['days_employed'].max()/24,'\n')

print('\nОсновные метрики стажа работающих:\n')
print('Минимум: ',abs(data[data['days_employed'] < 0]['days_employed'].max()))
print('Среднее: ',abs(data[data['days_employed'] < 0]['days_employed'].mean()))
print('Максимум:',abs(data[data['days_employed'] < 0]['days_employed'].min()),'\n')

display(data[data['days_employed'] < 0].sort_values(['days_employed']).head(10))
max_days_employed = data[data['days_employed'] < 0].sort_values(by = ['days_employed']).reset_index(drop=True)
for value in range(15):
    print(max_days_employed['days_employed'][value] / 365 + max_days_employed['dob_years'][value])


Основные метрики стажа пенсионеров и безработных после перевода часов в дни:

Минимум:  13697.03002518826
Среднее:  15208.51291317786
Максимум: 16739.80835313875 


Основные метрики стажа работающих:

Минимум:  24.14163324048118
Среднее:  2353.0159319988766
Максимум: 18388.949900568383 



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
3974,0,-15835.725775,64,среднее,1,гражданский брак,1,F,компаньон,0,96858.531436,сыграть свадьбу
1539,0,-15785.678893,59,высшее,0,Не женат / не замужем,4,F,сотрудник,0,119563.851852,операции с коммерческой недвижимостью
4321,0,-15773.061335,61,среднее,1,гражданский брак,1,F,сотрудник,0,205868.585780,свадьба
7731,0,-15618.063786,64,среднее,1,женат / замужем,0,F,компаньон,0,296525.358574,высшее образование
15675,0,-15410.040779,65,высшее,0,женат / замужем,0,F,сотрудник,0,188800.068859,покупка жилой недвижимости


10.619315340908543
12.738182833896133
14.538430637633375
14.43917944962432
19.83647195426324
20.614449932022183
15.751564675739715
17.786133328975616
21.210784147001107
22.780710194258965
14.171120047303809
15.37525424262163
13.687079264202318
15.123151219060645
27.594827577935902


Данные в целом в порядке, но если посмотреть на самые большие стажи, то можно увидеть клиентов которые начали работать минимум до достижения 11 лет, что возможно является ошибкой. Проведем преобразования.

In [8]:
display(data.head(5))

for value in range(len(data)):
    if data['days_employed'][value] > 0:
        data['days_employed'][value] = data['days_employed'][value].copy() / 24
    if data['days_employed'][value] < 0:
        data['days_employed'][value] = abs(data['days_employed'][value])
        
display(data.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [9]:
print('\nОсновные метрики стажа пенсионеров и безработных:\n')
print('Минимум: ',data[(data['income_type'] == 'пенсионер') | (data['income_type'] == 'безработный')]['days_employed'].min())
print('Среднее: ',data[(data['income_type'] == 'пенсионер') | (data['income_type'] == 'безработный')]['days_employed'].mean())
print('Максимум:',data[(data['income_type'] == 'пенсионер') | (data['income_type'] == 'безработный')]['days_employed'].max(),'\n')

print('\nОсновные метрики стажа работающих:\n')
print('Минимум: ',data[(data['income_type'] != 'пенсионер') & (data['income_type'] != 'безработный')]['days_employed'].min())
print('Среднее: ',data[(data['income_type'] != 'пенсионер') & (data['income_type'] != 'безработный')]['days_employed'].mean())
print('Максимум:',data[(data['income_type'] != 'пенсионер') & (data['income_type'] != 'безработный')]['days_employed'].max(),'\n')

print('\nОбщие основные метрики стажа:\n')
print('Минимум: ',data['days_employed'].min())
print('Среднее: ',data['days_employed'].mean())
print('Максимум:',data['days_employed'].max())
print('Медиана: ',data['days_employed'].median(),'\n')

print('\nОсновные метрики дохода:\n')
print('Минимум: ',data['total_income'].min())
print('Среднее: ',data['total_income'].mean())
print('Максимум:',data['total_income'].max())
print('Медиана: ',data['total_income'].median(),'\n')

display(data.sort_values(['total_income'], ascending=False).head(10))


Основные метрики стажа пенсионеров и безработных:

Минимум:  13697.03002518826
Среднее:  15208.512913177861
Максимум: 16739.80835313875 


Основные метрики стажа работающих:

Минимум:  24.14163324048118
Среднее:  2353.015931998877
Максимум: 18388.949900568383 


Общие основные метрики стажа:

Минимум:  24.14163324048118
Среднее:  4641.641176180656
Максимум: 18388.949900568383
Медиана:  2194.220566878695 


Основные метрики дохода:

Минимум:  20667.26379327158
Среднее:  167422.30220817294
Максимум: 2265604.028722744
Медиана:  145017.93753253992 



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
17503,0,2285.476482,43,среднее,1,женат / замужем,0,M,компаньон,0,1.597613e+06,операции с недвижимостью
18368,1,333.935516,41,ВЫСШЕЕ,0,гражданский брак,1,M,компаньон,0,1.551153e+06,свадьба
18353,1,3173.282035,41,высшее,0,Не женат / не замужем,4,F,компаньон,0,1.427934e+06,автомобиль
15268,1,10207.448165,64,высшее,0,в разводе,3,M,компаньон,0,1.350246e+06,жилье
11071,1,1851.200013,36,высшее,0,гражданский брак,1,F,сотрудник,0,1.286281e+06,покупка коммерческой недвижимости


Характерными значениями положи медианы, в стаже скорее всего есть выбросы у работающих(в доходе может и нет но так как пропусков относительно немного и разница медианы и среднего небольшая думаю сильно это погоду не испортит).

In [10]:
median_days_employed = data['days_employed'].median()
median_total_income = data['total_income'].median()

data['days_employed'] = data['days_employed'].fillna(median_days_employed)
data['total_income'] = data['total_income'].fillna(median_total_income)

In [11]:
print(data['days_employed'].mean())
print(data['total_income'].mean())
print(data.info())

4394.454537173804
165159.48739726353
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Замена типа данных

In [12]:
data['days_employed'] = data['days_employed'].astype('int')
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [13]:
#display(data.groupby('family_status')['family_status_id'].value_counts())

In [14]:
data.info()
data['children'] = data['children'].astype('int8')
data['days_employed'] = data['days_employed'].astype('int16')
data['dob_years'] = data['dob_years'].astype('int8')
data['education_id'] = data['education_id'].astype('int8')
data['family_status_id'] = data['family_status_id'].astype('int8')
data['total_income'] = data['total_income'].astype('float32')
display(data.head())
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.640625,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.015625,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.953125,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.562500,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.078125,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int16
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float32
purpose             21525 non-null object
dtypes: float32(1), int16(1), int64(1), int8(4), object(5)
memory usage: 1.2+ MB


### Обработка дубликатов

In [15]:
print(len(data) - len(data.drop_duplicates()))# проверяем количество явных дубликатов
data = data.drop_duplicates().reset_index(drop=True) # удаление дубликатов этим методом

for value in range(len(data)):
    data['education'][value] = data['education'][value].lower()
    data['family_status'][value] = data['family_status'][value].lower() # за компанию

print(data['education'].unique())
print(data['family_status'].unique())

54
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


In [16]:
print(data['children'].value_counts(),'\n')
print(data['dob_years'].unique(),'\n')
mean_dob_years = data['dob_years'].mean().astype('int')
data.info()
for value in range(len(data)):
    if (data['children'][value] == 20) | (data['children'][value] == -1):
        data['children'][value] = nan
    if (data['dob_years'][value] == 0):
        data['dob_years'][value] = mean_dob_years
median_children = data['children'].median()
data['children'] = data['children'].fillna(median_children).astype('int')

print('\n',data['dob_years'].unique(),'\n')

print(data['children'].value_counts(),'\n')
data.info()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64 

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
children            21471 non-null int8
days_employed       21471 non-null int16
dob_years           21471 non-null int8
education           21471 non-null object
education_id        21471 non-null int8
family_status       21471 non-null object
family_status_id    21471 non-null int8
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null float32
purpose             21471 non-null object
dtypes: float32(1), int16(1), int64(1), int8(4), object(5)
memory usage: 1.2+ MB


### Лемматизация

In [17]:
m = Mystem()

lemmas = m.lemmatize(' '.join(data['purpose']))
    
print(Counter(lemmas))

Counter({' ': 55066, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'подержать': 853, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


In [18]:
real_estate = ['недвижимость','жилье']
auto = 'автомобиль'
education = 'образование'
wedding = 'свадьба'
data['purpose_category'] = ''
for value in range(len(data)):
    if (real_estate[0] in m.lemmatize(data['purpose'][value])) or (real_estate[1] in m.lemmatize(data['purpose'][value])):
        data['purpose_category'][value] = 'недвижимость'
    elif auto in m.lemmatize(data['purpose'][value]):
        data['purpose_category'][value] = 'автомобиль'
    elif education in m.lemmatize(data['purpose'][value]):
        data['purpose_category'][value] = 'образование'
    elif wedding in m.lemmatize(data['purpose'][value]):
        data['purpose_category'][value] = 'свадьба'
    else:
        data['purpose_category'][value] = 'остальное'
display(data)
print(data['purpose_category'].unique())
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.640625,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.015625,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.953125,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.562500,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.078125,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.859375,операции с жильем,недвижимость
21467,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.812500,сделка с автомобилем,автомобиль
21468,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.562500,недвижимость,недвижимость
21469,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.046875,на покупку своего автомобиля,автомобиль


['недвижимость' 'автомобиль' 'образование' 'свадьба']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 13 columns):
children            21471 non-null int64
days_employed       21471 non-null int16
dob_years           21471 non-null int8
education           21471 non-null object
education_id        21471 non-null int8
family_status       21471 non-null object
family_status_id    21471 non-null int8
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null float32
purpose             21471 non-null object
purpose_category    21471 non-null object
dtypes: float32(1), int16(1), int64(2), int8(3), object(6)
memory usage: 1.5+ MB


In [19]:
display(data)
print(data['purpose_category'].unique())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.640625,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.015625,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.953125,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.562500,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.078125,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.859375,операции с жильем,недвижимость
21467,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.812500,сделка с автомобилем,автомобиль
21468,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.562500,недвижимость,недвижимость
21469,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.046875,на покупку своего автомобиля,автомобиль


['недвижимость' 'автомобиль' 'образование' 'свадьба']


**Вывод**

Подключили Mystem для лемматизации, Counter для подсчета вхождений, слемматизировали, подсчитали, выделили самые частые группы, пробежлись с проверкой по фрейму записали группы.

### Категоризация данных

In [20]:
def income_category(income):
    if income <= 120000:
        return 'низкий доход'
    elif 300 < income <= 200000:
        return 'средний доход'
    elif income > 200000:
        return 'высокий доход'

data['income_category'] = data['total_income'].apply(income_category)

display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.640625,покупка жилья,недвижимость,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.015625,приобретение автомобиля,автомобиль,низкий доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.953125,покупка жилья,недвижимость,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.562500,дополнительное образование,образование,высокий доход
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.078125,сыграть свадьбу,свадьба,средний доход
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.859375,операции с жильем,недвижимость,высокий доход
21467,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.812500,сделка с автомобилем,автомобиль,средний доход
21468,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.562500,недвижимость,недвижимость,низкий доход
21469,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.046875,на покупку своего автомобиля,автомобиль,высокий доход


**Вывод**

приблизительно поделил выборку на три части, меньше 120000 - низкий доход, меньше 200000 - средний, все что больше высокий.

In [21]:
display(data.groupby('income_category')['total_income'].count())

income_category
высокий доход    5066
низкий доход     6845
средний доход    9560
Name: total_income, dtype: int64

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
children = data.groupby('children')['debt'].value_counts()
display(children)
print('\n% невозвратов если нет детей:',1072/(1072+13158))
print('% невозвратов если есть дети:',(444 + 194 + 27 + 4)/children[2:11].sum())
print('\n---------------------------------------\n')
print('% невозвратов если один ребенок:',444/(444 + 4365))
print('% невозвратов если детей >2:    ',(194 + 27 + 4)/children[4:11].sum())

children  debt
0         0       13158
          1        1072
1         0        4365
          1         444
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64


% невозвратов если нет детей: 0.07533380182712579
% невозвратов если есть дети: 0.09239055379091286

---------------------------------------

% невозвратов если один ребенок: 0.09232688708671241
% невозвратов если детей >2:     0.09251644736842106


**Вывод**

Разница невозвратов пол процента в сторону клиентов с детьми(можно сказать не существенная)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
family = data.groupby('family_status')['debt'].value_counts()
display(family)
print('\nпроценты невозвратов:\n')
print('в разводе             ',85/(85+1110))
print('вдовец / вдова        ',63/(63+896))
print('гражданский брак      ',388/(388+3775))
print('женат / замужем       ',931/(931+11413))
print('не женат / не замужем ',274/(274+2536))

family_status          debt
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3775
                       1         388
женат / замужем        0       11413
                       1         931
не женат / не замужем  0        2536
                       1         274
Name: debt, dtype: int64


проценты невозвратов:

в разводе              0.07112970711297072
вдовец / вдова         0.06569343065693431
гражданский брак       0.09320201777564256
женат / замужем        0.07542125729099157
не женат / не замужем  0.09750889679715302


**Вывод**

Четкий ответ дать сложно разница между группами небольшая, и сами группы сильно отличаются по размеру(количество вдов/вдовцов к примеру сильно меньше женат/не замужем), но наблюдаеься тенденция что люди не вступающие в оффициальный брак чаще имеют просрочку по кредиту

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
income = data.groupby('income_category')['debt'].value_counts()
display(income)

print('\nнизкий доход:  ',551/(551+6294))
print('\nсредний доход: ',832/(832+8728))
print('\nвысокий доход: ',358/(358+4708))

income_category  debt
высокий доход    0       4708
                 1        358
низкий доход     0       6294
                 1        551
средний доход    0       8728
                 1        832
Name: debt, dtype: int64


низкий доход:   0.08049671292914536

средний доход:  0.08702928870292886

высокий доход:  0.07066719305171733


**Вывод**

Разница не большая, можно выделлить что люди с высоким доходом возвращают охотнее

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
purpose = data.groupby('purpose_category')['debt'].value_counts()
display(purpose)

print('\nавтомобиль:  ',403/(403+3905))
print('\nнедвижимость:',782/(782+10032))
print('\nобразование: ',370/(370+3644))
print('\nсвадьба    : ',186/(186+2149))

purpose_category  debt
автомобиль        0        3905
                  1         403
недвижимость      0       10032
                  1         782
образование       0        3644
                  1         370
свадьба           0        2149
                  1         186
Name: debt, dtype: int64


автомобиль:   0.09354688950789229

недвижимость: 0.0723136674680969

образование:  0.09217737917289487

свадьба    :  0.07965738758029979


**Вывод**

Люди берущие кредиты на покупку/постройку/ремонт/какие либо еще операции с недвижимостью самые надежные клиенты(по факту опять сделать однозначный вывод сложно данные не равномерно распределены)

## Шаг 4. Общий вывод

Были исследованы данные, в них обнаружены:
1. Разного рода пропуски и шумы.
2. Стаж безработных и пенсионеров по какой то причине измеряется в часах, а стажи всех остальных имеют отрицательные значения.

Ответы на вопросы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть, при наличии детей вероятность просрочить выплату увеличивается с 7.5% до 9.2% что составляет ~1.7%
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

    - Есть, для групп были полученны следующие вероятности просрочить выплату:

        - в разводе - 7.11%

        - вдовец / вдова - 6.57%

        - гражданский брак - 9.32%

        - женат / замужем - 7.54%

        - не женат / не замужем - 9.75%

        - Можно выделить что люди заключающие оффициальный брак(и состоящие в нем ранее) имеют вероятность просрочить выплату на ~2.47% меньше (7.07% против 9.54% соответственно)

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Есть, но не ярко выраженная, люди с доходами выше среднего имеют вероятность просрочить выплату на ~1% меньше(7.1% против 8.35%)

4. Как разные цели кредита влияют на его возврат в срок?
    - Есть, для групп были полученны следующие вероятности просрочить выплату:

---
автомобиль - 9.35%

недвижимость - 7.23%

образование - 9.22%

свадьба - 7.97%

---

- Люди, берущие кредит на недвижимость и свадьбы, имеют вероятность просрочить кредит на ~1.69% ниже, чем люди берущие кредит на покупку авто и оплату образования (7.6% против 9.26%)

---

П.с: группы не достаточно большие и неравномерные, стоит провести анализ на выборке побольше.